In [1]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SeqIO 
import pandas as pd
from Bio import Entrez
import matplotlib.pyplot as plt

# Import the sequence(s)

In [4]:
seq_record = SeqIO.read('sequences/test/sequence_test.fa','fasta')
#seq_record.id 
seq_record.seq 

Seq('CCTCACATACCAGATGCGAATTCCCGAG', SingleLetterAlphabet())

## Get the complement sequence

In [2]:
def complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A','a': 't', 'c': 'g', 'g': 'c', 't': 'a'} 
    bases = list(seq) 
    bases = [complement[base] for base in bases] 
    return ''.join(bases)

## Multiple sequences

In [25]:
sequ = list(SeqIO.parse('sequences/pathogens/16s_tox_genes_oyster_genes.fa','fasta'))
sequ[0].seq

seq = []

for i in sequ:
    seq.append(complement(i.seq))
    
print(len(sequ))

6


In [34]:
seq[1]

'GCGCATGCAGGTGGTTCGTTAAGTCAGA'

In [26]:
df = pd.DataFrame()

df['ID'] = [sequ[i].id for i in range(len(seq))]
df['Seq'] = [seq[i] for i in range(len(seq))]

df.iloc[2]

ID                        Guide_VAM
Seq    ataaaggtttgcctgtttttaatacttc
Name: 2, dtype: object

# Call blast from the NCBI website

In [27]:
result_handle = []

Entrez.email = 'ness.louafi@gmail.com'

for i in range(len(seq)):
    print('Trying sequence number ' + str(i)+'...')
    result_handle.append(NCBIWWW.qblast("blastn", "nt", seq[i], hitlist_size = 20,expect=0.05))
    print('Done with sequence ' + str(i) + ' !')
    

Trying sequence number 0...
Done with sequence 0 !
Trying sequence number 1...
Done with sequence 1 !
Trying sequence number 2...
Done with sequence 2 !
Trying sequence number 3...
Done with sequence 3 !
Trying sequence number 4...
Done with sequence 4 !
Trying sequence number 5...
Done with sequence 5 !


In [16]:
NCBIWWW.qblast?



# Display the results

In [28]:
'''
for i in range(len(result_handle)):
    with open('results/05072022/16s_tox_genes_oyster_genes_'+str(i)+'.xml', 'w') as save_file: 
        blast_results = result_handle[i].read() 
        save_file.write(blast_results)
'''     

for i in range(len(seq)):
    for record in NCBIXML.parse(open("results/05072022/16s_tox_genes_oyster_genes_"+str(i)+".xml")): 
         if record.alignments: 
            print("\n") 
            print(df['ID'][i])
            #print("results"+str(i))
            for align in record.alignments: 
                for hsp in align.hsps: 
                    if hsp.expect < 0.05:

                        print("match: %s " % align.title[:120])
                        print('e value:', hsp.expect)
                        
                    #print('e value:', hsp.expect)
'''          
    for alignment in record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < 0.05:
                print('****Alignment****')
                #print('length:', alignment.length)
                print('e value:', hsp.expect)
                #print(hsp.query[0:75] + '   ')
                #print(hsp.match[0:75] + '   ')
                #print(hsp.sbjct[0:75] + '   ')
'''           



Guide_16s01
match: gi|1815532793|gb|MT125688.1| Vibrio sp. strain SCA-132 16S ribosomal RNA gene, partial sequence 
e value: 0.000627875
match: gi|1809665820|gb|MT071165.1| Vibrio metschnikovii strain SCA-95 16S ribosomal RNA gene, partial sequence 
e value: 0.000627875
match: gi|1777175393|gb|MN713490.1| Vibrio metschnikovii strain 486 16S ribosomal RNA gene, partial sequence 
e value: 0.000627875
match: gi|1777175370|gb|MN713467.1| Vibrio metschnikovii strain 446 16S ribosomal RNA gene, partial sequence 
e value: 0.000627875
match: gi|1777175368|gb|MN713465.1| Vibrio metschnikovii strain 436 16S ribosomal RNA gene, partial sequence 
e value: 0.000627875
match: gi|1784008151|gb|MN826338.1| Vibrio metschnikovii strain Vm 16S ribosomal RNA gene, partial sequence 
e value: 0.000627875
match: gi|1785416944|gb|CP046822.1| Vibrio metschnikovii strain 07-2421 chromosome 1, complete sequence 
e value: 0.000627875
match: gi|1785416944|gb|CP046822.1| Vibrio metschnikovii strain 07-2421 chromo

"          \n    for alignment in record.alignments:\n        for hsp in alignment.hsps:\n            if hsp.expect < 0.05:\n                print('****Alignment****')\n                #print('length:', alignment.length)\n                print('e value:', hsp.expect)\n                #print(hsp.query[0:75] + '   ')\n                #print(hsp.match[0:75] + '   ')\n                #print(hsp.sbjct[0:75] + '   ')\n"

Put all the rsult per query in a dataframe (use the one defined above) and then find a way to characterize the species of each hit to try and filter the results.

The impression is that many sequences have 16s hits.... 

In [14]:
df["Matches"] = ""

for i in range(15):
    for record in NCBIXML.parse(open("results/vibrio_16s/corrected_analysis/corrected_"+str(i)+".xml")): 
        align_organism = []
        e_val = []
        for align in record.alignments: 
            #print(e_val)
            align_organism.append(align.hit_def)
            e_val.append(align.hsps.expect)
        print(e_val)
        #df.at[i,"Matches"] = align_organism
        #df.at[i,"e_Value"] = e_val
df

AttributeError: 'list' object has no attribute 'expect'

In [9]:
df

,ID,Seq,Matches,e_Value
0,G_01,ggtaacatttcaaaagcttgcttttgaa,[Shewanella algae strain ROD067 16S ribosomal ...,NaN
1,G_02,gttgtaaagtactttcagtcgtgaggaa,,NaN
2,G_03,gggtagaatttcaggtgtagcggtgaaa,,NaN
3,G_04,ggggataaccattggaaacgatggctaa,,NaN
4,G_05,tattgcacaatgggcgcaagcctgatgc,,NaN
5,G_06,gtattctttgacgttagcgacagaagaa,,NaN
6,G_07,tgatagaaatcaaggggatgcaagcgct,,NaN
7,G_08,cttatgatcaatatggtcatgcagcctt,,NaN
8,G_09,acattttcggtgatgtctttggtgatat,,NaN
9,G_10,acgctacaacatggaattatcgctagaa,,NaN


Now the idea is for each match, take the organism (class) from the NCBI accession and then be able to plot the kind of output we got per sequence 

In [55]:
align_organism

Entrez.email = 'ness.louafi@gmail.com'

handle = Entrez.esearch(db="nt",accession=align_organism[0])
print(handle)

TypeError: esearch() missing 1 required positional argument: 'term'

In [ ]:
help(NCBIWWW.qblast)

In [29]:
from Bio import Entrez
from urllib.error import URLError
import time

Entrez.email = 'ness.louafi@gmail.com'
counter = 0 
list_record_host = []
for record in SeqIO.parse("sequences/test/sequences_10.fa", format="fasta"):
    print(record.id)
    print(counter)
    counter+=1
#         print(record.seq)

    # online request
    try:
        result_handle = NCBIWWW.qblast("blastn","nt",record.seq, hitlist_size = 10)
        print(result_handle)
    except HTTPError:
        time.sleep(5)
        result_handle = NCBIWWW.qblast("blastn","nt",record.seq, hitlist_size = 10)

    # result handle stored in a list
    list_record_host.append(result_handle)

0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9


In [30]:
for i in range(len(list_record_host)):
    with open('results/results'+str(i)+'.xml', 'w') as save_file: 
        blast_results = result_handle.read() 
        save_file.write(blast_results)

# Save and open the results

In [ ]:
for i in range(50):
    with open('results'+str(i)+'.xml', 'w') as save_file: 
        blast_results = result_handle.read() 
        save_file.write(blast_results)

E_VALUE_THRESH = 1e-20 
for record in NCBIXML.parse(open("results.xml")): 
     if record.alignments: 
        print("\n") 
        print("query: %s" % record.query[:100]) 
        for align in record.alignments: 
            for hsp in align.hsps: 
                if hsp.expect < E_VALUE_THRESH: 
                    print("match: %s " % align.title[:100])

In [ ]:
blast_params = {'program': 'blastp', 'database': 'nr', 'sequence': seq_record.seq, 'expect': 10.0}
blast_params['database'] = ['nr']
print_data = pd.DataFrame()
for database in blast_params['database']:
    db_values = {}
    result = NCBIWWW.qblast(blast_params['program'], database, blast_params['sequence'], expect=blast_params['expect'])
    file_name = "blast_output_" + database + ".xml"
    with open(file_name, "w") as output_xml:
        output_xml.write(result.read())
    result.close()
    result_input = open(file_name)
    blast_records = NCBIXML.read(result_input)
    for description in blast_records.descriptions:
        if 'score' in db_values:
            db_values['score'].append(description.score)
        else:
            db_values['score'] = [description.score]
        if 'e-value' in db_values:
            db_values['e-value'].append(description.e)
        else:
            db_values['e-value'] = [description.e]
    df = pd.DataFrame.from_dict(db_values)
    df['database'] = database[0:6] # we simply limit the name to the first 6 characters for easier viewing
    frames = [print_data, df]
    print_data = pd.concat(frames, ignore_index=True)
    

In [ ]:
E_VALUE_THRESH = 1e-20 
for record in NCBIXML.parse(open("blast_output_nr.xml")): 
     if record.alignments: 
        print("\n") 
        print("query: %s" % record.query[:100]) 
        for align in record.alignments: 
            for hsp in align.hsps: 
                if hsp.expect < E_VALUE_THRESH:
                    if "Arabidopsis thaliana" in align.title:
                        print("match: %s " % align.title[:100])

# Open when finished

In [ ]:
import webbrowser
webbrowser.open_new('http://localhost:8888/notebooks/Documents/iGEM/iGEM/Run%20BLAST.ipynb')